## LangGraph / LangChain 

### Install dependencies

```shell
uv add langgraph
uv add langchain-openai
```

### Other dependencies
```shell
uv add typing
uv add pydantic
```

In [ ]:
# Imports

from langgraph.graph import StateGraph, START, END
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from IPython.display import Image, display
from typing import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages 

### Try out ChatOpenAI LLM from LangChain

In [ ]:
## Using LLM through ChatOpenAI
## Make sure you have OPENAI_API_KEY set in your environment variables

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# Invoke the LLM
response = llm.invoke("What is the capital of Australia?")
print(response.content)

## Building the Graph


**Step 1: Define the state**

We need the `add_messages` reducer function so that each state transition will append the previous messages

In [ ]:
# Step 1: Define the state object 
# 
# A state could be represented by either 
# a `pydantic` model or a simple dictionary like `TypedDict`.

class State(TypedDict):
    messages: Annotated[list, add_messages]


**Step 2: Create the graph builder**

In [ ]:
# New graph builder
state_graph_builder = StateGraph(State)

**Step 3: Create nodes**

In [ ]:
# A node is represented by a function that takes in the old state and returns a new state
def first_node(old_state: State) -> State:
    return {"messages": [llm.invoke(old_state["messages"])]}


# Add this node to the graph
state_graph_builder.add_node(first_node)

**Step 4: Create edges**



In [ ]:
state_graph_builder.add_edge(START, "first_node")
state_graph_builder.add_edge("first_node", END)

**Step 5: Build the graph**

In [ ]:
graph = state_graph_builder.compile()

In [ ]:
# Display the graph
display(Image(graph.get_graph().draw_mermaid_png()))

**Step 6: Run the graph**

In [ ]:
prompt = "What is the capital of New Zealand?"
initial_state = State(messages=[{"role": "user", "content": prompt}])
result = graph.invoke(initial_state)

In [ ]:
# Simple message display without JSON
for i, msg in enumerate(result["messages"]):
    role: str = getattr(msg, 'type', 'unknown')
    content: str = getattr(msg, 'content', str(msg))
    print(f"Message {i + 1} ({role}): {content}")